In [1]:
import agents
from dotenv import load_dotenv
# from openai import OpenAI
# openai_client = OpenAI()
load_dotenv()



True

In [2]:
from agents import Agent, function_tool, Runner
from agents import Runner

In [3]:
runner = Runner()
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

### Summary Agent


In [4]:
# Youtube video

import youtube

subtitles = youtube.fetch_youtube_transcript('vK_SxyqIfwk')
print(subtitles[:500])


0:00 Hey everyone, welcome to our event. This
0:02 event is brought to you by data talks
0:03 club which is a community of people who
0:05 love data. We have weekly events today.
0:08 Uh this is one of such events. Um if you
0:11 want to find out more about the events
0:13 we have, there is a link in the
0:14 description. Um so click on that link,
0:16 check it out right now. We actually have
0:19 quite a few events in our pipeline, but
0:21 we need to put them on the website. Uh
0:24 but keep a


In [5]:
summary_instructions = """
You're a helpful assistant that helps answer user questions
about YouTube videos
"""

In [6]:
summary_agent = Agent(
    name='summary_agent',
    instructions=summary_instructions,
    handoff_description="Whenever the user needs a summary of the video",
    tools=[function_tool(youtube.fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

### Search Agent

In [7]:
from minsearch import AppendableIndex

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
import docs

In [9]:
index = AppendableIndex(text_fields=['content'])

In [10]:
# Create the function

In [11]:
from typing import Any, Dict, List, Optional
from requests.exceptions import RequestException

# how do you know which agent are you going to create, and what is the function needed


class SearchTools:

    def __init__(self, index: Any) -> None:
        self.index = index

    def index_video(self, video_id: str, content: Optional[str] = None) -> bool:
        """
        Fetch and index the transcript of a YouTube video.

        Args:
            video_id (str): The unique identifier of the YouTube video.

        Returns:
            bool: True if indexing succeeds, False otherwise.
        """
        try:
            if not content:
                content = youtube.fetch_youtube_transcript(video_id)
                if not content:
                    print(f"No subtitles found for video: {video_id}")
                    return False

            chunks = docs.sliding_window(content, 2000, 1000)
            for chunk in chunks:
                chunk["video_id"] = video_id
                self.index.append(chunk)

            return True

        except (RequestException, ValueError) as e:
            print(f"Error indexing video {video_id}: {e}")
            return False
        except AttributeError:
            print("Error: 'youtube' or 'docs' module not properly configured.")
            return False
        except Exception as e:
            print(f"Unexpected error during indexing: {e}")
            return False

    def search(self, query: str) -> Dict[str, Any]:
        """
        Search for relevant results in the indexed video transcripts.

        Args:
            query (str): The user's search query.

        Returns:
            Dict[str, Any]: A dictionary containing the top search results.
        """
        if not isinstance(query, str) or not query.strip():
            raise ValueError("Query must be a non-empty string.")

        try:
            results = self.index.search(query, num_results=5)
            return results
        except AttributeError:
            raise AttributeError("Index object must implement a 'search(query, num_results=...)' method.")
        except Exception as e:
            print(f"Error performing search: {e}")
            return {}

In [12]:
search_tools = SearchTools(index)

In [13]:
from toyaikit.tools import wrap_instance_methods

search_method_tools = wrap_instance_methods(function_tool, search_tools)

In [14]:
# search_method_tools = [function_tool(search_tools.index_video), 
#                        function_tool(search_tools.search)
#                       ]

In [15]:
# search_tools.index_video('vK_SxyqIfwk')

In [16]:
# search_tools.search('tesla')

In [17]:
search_instructions = """
Your task is to search through indexed documents.

Before performing a search:

1. Check if the document has been indexed.
2. If not, call `index_video` to index it.

Important rules:

- Do NOT call `search` using a YouTube ID, webpage URL, or any direct link as the query.
- After successfully indexing a source, do NOT perform a search automatically.
- Instead, tell the user something like:
  "The video (or page) has been indexed. What would you like to learn about it?"

Only perform a search if the user provides a natural-language question or topic of interest
(e.g., "What does the video say about AI safety?"), not a URL or ID.

Do not attempt to summarize content.
"""

search_agent = Agent(
    name='search_agent',
    instructions=summary_instructions,
    handoff_description="Whenever the user needs a summary of the video",
    tools=search_method_tools,
    model='gpt-4o-mini'
)

In [18]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=search_agent
)

In [19]:
await runner.run();

You: stop


Chat ended.


### Summary Agent Updated

In [28]:
import youtube

In [33]:
class SummaryTools:

    def __init__(self, search_tools):
        self.search_tools = search_tools

    def fetch_youtube_transcript(self, video_id: str) -> str:
        """
        Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.
    
        Args:
            video_id (str): The unique YouTube video ID.
    
        Returns:
            str: The subtitles generated from the video's transcript.
        """
        content = youtube.fetch_youtube_transcript(video_id)
        self.search_tools.index_video(video_id, content)
        return content

In [30]:
summary_tools = SummaryTools(search_tools)

In [31]:
summary_instructions = """
You're a helpful assistant that summarizes youtube videos
"""

In [32]:
summary_agent = Agent(
    name='summary_agent',
    instructions=summary_instructions,
    handoff_description="Whenever the user needs a summary of the video",
    tools=[function_tool(summary_tools.fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

In [49]:
from agents import Runner
runner = Runner()

result = await runner.run(summary_agent, input='What is this video about https://www.youtube.com/watch?v=ZFrcrTtnB1Q?')

In [61]:
result.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"video_id":"ZFrcrTtnB1Q"}', call_id='call_4XeMqlcuQVgu6CKUNQeNJQbN', name='fetch_youtube_transcript', type='function_call', id='fc_07b8895ee59a73930068f1bc0bb83c8196b4508f27efeb64ba', status='completed')

### Triage / Orchestrator

In [23]:
from agents import handoff

In [34]:
triage_instructions = """
You are the orchestrator between two specialized agents:

1. summarizing_agent — summarizes web pages or YouTube videos.
2. search_agent — searches within previously indexed documents to answer detailed or follow-up questions.

Routing rules:

- If the user sends a YouTube link without any question → hand off to summary_agent.
- If the user asks for a summary, overview, or "what is this video about" → summary_agent.
- If the user asks a follow-up question after a summary (e.g. "how did they do X?", "what does she say about Y?"),
  or refers to something mentioned in a previously summarized or indexed resource → hand off to search_agent.
- If the user asks a direct content question about a topic inside a resource → search_agent.
- Prefer delegating the answer to agents when possible. 

Examples:
User: "https://youtube.com/watch?v=abc123"
→ summary_agent

User: "What does the video say about climate change?"
→ search_agent

User: "How exactly did they fight malaria?" (after a summary)
→ search_agent
""".strip()

triage_agent = Agent(
    name='triage_agent',
    instructions=triage_instructions,
    handoffs=[
        handoff(summary_agent, on_handoff=lambda ctx: print('handoff to summary agent')),
        handoff(search_agent, on_handoff=lambda ctx: print('handoff to search agent')),
    ],
    model='gpt-4o-mini'
)

In [35]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=triage_agent
)

In [37]:
"""
handoff to summary agent
handoff: transfer_to_summary_agent
handoff: summary_agent -> triage_agent successful

Question - how does this handoff to each other? - toyaikit package

"""

'\nhandoff to summary agent\nhandoff: transfer_to_summary_agent\nhandoff: summary_agent -> triage_agent successful\n\nQuestion - how does this handoff to each other?\n\n'

In [36]:
await runner.run();

You: What is this video about https://www.youtube.com/watch?v=ZFrcrTtnB1Q?


handoff to summary agent
handoff: transfer_to_summary_agent
handoff: summary_agent -> triage_agent successful


You: stop


Chat ended.
